### Can we classify each phase as relatively low or high affect for each subject? ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)

import tools.data_reader_popane as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


c:\Users\zhoux\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

`use_label_encoder` is deprecated in 1.7.0.


In [8]:
# STUDY 1
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

studies = [
    "Study1",
    "Study2",
    "Study3",
    "Study5"
]
phases = [
    dr.Study1.ALL,
    dr.Study2.ALL,
    dr.Study3.ALL,
    dr.Study5.ALL
]
label_type = "affect"

for i, study in enumerate(studies):
    print(f"{study} --------------------")
    x, y = train.Train_POPANE.get_popane_data(study, metrics, phases[i], verbose=False, label_type=label_type)
    inds = pd.isnull(x).any(1).to_numpy().nonzero()[0]
    x = x.drop(inds, axis=0)
    x = x.drop(["phaseId"], axis=1)
    y = y.drop(inds, axis=0)

    print(y["label"].value_counts())
    print("")

    # 0-1 scaling
    for i in range(2, len(x.columns)):
        if x.columns[i] in metrics:
            data_col = x[x.columns[i]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x[x.columns[i]] = data_col

    acc_results = {
            "SVM": [],
            "LogReg": [],
            "XGB": []
    }
    reports = {
            "SVM": [],
            "LogReg": [],
            "XGB": []
    }
    num_iters = 10
    for _ in range(num_iters):
        out = train.train_predict(models, x, y, test_size=0.15, by_subject=False, save_metrics=True, get_shap_values=False)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0.65:
            print(f"Model evaluation metrics for {model_name}:")
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("")

    print("\n")

Study1 --------------------


In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.


1    215
0    199
Name: label, dtype: int64

SVM accuracy over 10 rounds: 0.3027146130794994

LogReg accuracy over 10 rounds: 0.48103472589458834

XGB accuracy over 10 rounds: 0.4790490040542923



Study2 --------------------


In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.


1    272
0    264
Name: label, dtype: int64

SVM accuracy over 10 rounds: 0.3382187085066832

LogReg accuracy over 10 rounds: 0.5397629796363974

XGB accuracy over 10 rounds: 0.4753946802143004



Study3 --------------------


In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.


1    220
0    197
Name: label, dtype: int64

SVM accuracy over 10 rounds: 0.3999136259474705

LogReg accuracy over 10 rounds: 0.5106575004406839

XGB accuracy over 10 rounds: 0.42749779658029263



Study5 --------------------


In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.


1    314
0    275
Name: label, dtype: int64

SVM accuracy over 10 rounds: 0.31797465432188765

LogReg accuracy over 10 rounds: 0.5103721659254938

XGB accuracy over 10 rounds: 0.5343412067750479





In [9]:
# STUDY 1, 2, 3, 5 all together
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

label_type = "affect"

study = "Study1"
phases = dr.Study1.ALL

x1, y1 = train.Train_POPANE.get_popane_data(study, metrics, phases, verbose=False, label_type=label_type)
inds = pd.isnull(x1).any(1).to_numpy().nonzero()[0]
x1 = x1.drop(inds, axis=0)
y1 = y1.drop(inds, axis=0)

study = "Study2"
phases = dr.Study2.ALL

x2, y2 = train.Train_POPANE.get_popane_data(study, metrics, phases, verbose=False, label_type=label_type)
inds = pd.isnull(x2).any(1).to_numpy().nonzero()[0]
x2 = x2.drop(inds, axis=0)
y2 = y2.drop(inds, axis=0)

study = "Study3"
phases = dr.Study3.ALL

x3, y3 = train.Train_POPANE.get_popane_data(study, metrics, phases, verbose=False, label_type=label_type)
inds = pd.isnull(x3).any(1).to_numpy().nonzero()[0]
x3 = x3.drop(inds, axis=0)
y3 = y3.drop(inds, axis=0)

study = "Study5"
phases = dr.Study5.ALL

x5, y5 = train.Train_POPANE.get_popane_data(study, metrics, phases, verbose=False, label_type=label_type)
inds = pd.isnull(x5).any(1).to_numpy().nonzero()[0]
x5 = x5.drop(inds, axis=0)
y5 = y5.drop(inds, axis=0)

# print(x1.shape)
# print(x3.shape)
# print(x4.shape)
# print(x5.shape)

x = pd.concat([x1, x2, x3, x5])
x = x.drop(["phaseId"], axis=1)
y = pd.concat([y1, y2, y3, y5])

# print(x.head())
# print(x.shape)

print(y["label"].value_counts())
print("")

# 0-1 scaling
for i in range(2, len(x.columns)):
    if x.columns[i] in metrics:
        data_col = x[x.columns[i]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x[x.columns[i]] = data_col

acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
}
reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.train_predict(models, x, y, by_subject=False, save_metrics=True, get_shap_values=False)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.65:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("")

In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.


1    1021
0     935
Name: label, dtype: int64

SVM accuracy over 10 rounds: 0.37212547344939895

LogReg accuracy over 10 rounds: 0.5284400523407256

XGB accuracy over 10 rounds: 0.4914417388817869

